In [1]:
import numpy as np
from scipy.stats import t
import pandas as pd

In [2]:
####### get feature table and fscores for each fold

featureTable_c4_raw = pd.read_csv("../../feature/FeatureTables/featureTable_combinedSelected_halfLifeClass/HalfLifeCls_hypoxia_combinedSelected_leadered_byLeaderedLogPhase_jointlyByLogPhaseHypoxia.csv", header = 0)
scores_raw_p1 = pd.read_csv('../ml_metrics_halfLifeClass/HalfLifeCls_hypoxia_combinedSelected_leadered_byLeaderedLogPhase_jointlyByLogPhaseHypoxia_fscore_allFold.txt', header = None, sep = " ")
scores_raw_p2 = scores_raw_p1.iloc[:2, 1:]


In [3]:
####### corrects standard deviation

def corrected_std(differences, n_train, n_test):
    """Corrects standard deviation using Nadeau and Bengio's approach.

    Parameters
    ----------
    differences : ndarray of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    corrected_std : float
        Variance-corrected standard deviation of the set of differences.
    """
    # kr = k times r, r times repeated k-fold crossvalidation,
    # kr equals the number of times the model was evaluated
    kr = len(differences)
    corrected_var = np.var(differences, ddof = 1) * (1 / kr + n_test / n_train)
    corrected_std = np.sqrt(corrected_var)
    return corrected_std


In [4]:
####### get Nadeau and Bengio’s corrected t-test

def compute_corrected_ttest(differences, df, n_train, n_test):
    """Computes right-tailed paired t-test with corrected variance.

    Parameters
    ----------
    differences : array-like of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    df : int
        Degrees of freedom.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    t_stat : float
        Variance-corrected t-statistic.
    p_val : float
        Variance-corrected p-value.
    """
    mean = np.mean(differences)
    std = corrected_std(differences, n_train, n_test)
    t_stat = mean / std
    p_val = t.sf(np.abs(t_stat), df)  # right-tailed t-test
    return t_stat, p_val


In [5]:
####### pvalues of Nadeau and Bengio’s corrected t-test

scores_toCompare = scores_raw_p2.to_numpy()
differences = scores_toCompare[0] - scores_toCompare[1]
n = differences.shape[0]  # number of test sets
df = n - 1
n_train = 4 * featureTable_c4_raw.shape[0] / 5
n_test = featureTable_c4_raw.shape[0] / 5

t_stat, p_val = compute_corrected_ttest(differences, df, n_train, n_test)
print(f"Corrected t-value: {t_stat:.3f}\nCorrected p-value: {p_val}")


Corrected t-value: 4.507
Corrected p-value: 2.0510534280346034e-05
